This is the second part of the scraping workflow. Here, I take the results of the scraping and parse them to a dataframe and csv to export. 

In [1]:
#Read in files 
import pickle 

with open('AntipsychoticMedicationUseResults.pkl', 'rb') as f:
  antiPsy = pickle.load(f)
with open('FallsResults.pkl', 'rb') as f:
  falls = pickle.load(f)
with open('PhysicalRestraintsUseResults.pkl', 'rb') as f:
  physRes = pickle.load(f)
with open('PressureUlcersResults.pkl', 'rb') as f:
  pressureUl = pickle.load(f)
with open('depressionResults.pkl', 'rb') as f:
  depression = pickle.load(f)
with open('painResults.pkl', 'rb') as f:
  pain = pickle.load(f)


In [2]:
#Make into induvidual dfs 

import pandas as pd 

homesList = []
for ltc in pain: 
    name = ltc["DataTableCollection"][0]["OrgName"]
    homesList.append(name.lower())

def getParsed(indlcator):
  toReturn = []
  for ltc in indlcator: 
    measure = ltc["DataTableCollection"][0]["MeasureColumn"]
    toReturn.append(measure)
  return toReturn

#Parse
painList = getParsed(pain)
depressionList = getParsed(depression)
pressureUlList = getParsed(pressureUl)
physResList = getParsed(physRes)
fallsList = getParsed(falls)
antiPsyList = getParsed(antiPsy)



#Make into df 
toDict = {'Home': homesList, 'Pain': painList, 'Depression': depressionList, 'PressureUlcers': pressureUlList, 'PhysicalRestraintUse': physResList, 'Falls': fallsList, 'AntiPhsychoticMedUse': antiPsyList} 
df = pd.DataFrame(toDict)


In [25]:
df.to_csv('HomePerformanceMetrics.csv', index = False)

In [ ]:
df2 = pd.read_csv("CombinedLTCFixed.csv")
#attempt = pd.merge(df2, df,  how='left', left_on="LTC_Home", right_on="Home")
original = list(df2["LTC_Home"])
for i in range(len(original)):
  original[i] = original[i].lower()

sameList = []
for home1 in original:
  for home2 in homesList:
    if (home1.lower().strip()==home2.lower().strip()):
      sameList.append(home2)

lookList = []
for item in original:
  if (item not in sameList):
    lookList.append(item)

print("We have", len(sameList), "We still need", len(lookList))

#Try matching first word       
testDict = {}
for item in lookList:
  itemCheck  = item.split(" ")[0]
  for item2 in homesList:
    item2Check = item2.split(" ")[0]
    if(itemCheck ==item2Check):
      testDict[item] = item2

#Get chartwells
chartwellsOriginal = []
chartwellsScraped = []
for item in lookList: 
  if (item.split(" ")[0] == "chartwell"):
    chartwellsOriginal.append(item)
for item in homesList:
  if (item.split(" ")[0] == "chartwell"):
    chartwellsScraped.append(item)
chartwellsOriginal.sort()
chartwellsScraped.sort()

fixedChartwell = []
finalDict = {}
for item in chartwellsOriginal:
  originalC = item.split(" ")[1]
  for item2 in chartwellsScraped:
    scraped = item2.split(" ")[1]
    if (originalC == scraped):
      finalDict[item] = item2
      fixedChartwell.append(item)

sameList = sameList + fixedChartwell 
lookList = []
for item in original:
  if (item not in sameList):
    lookList.append(item)

print("We have", len(sameList), "We still need", len(lookList), "original was", len(original))

#Try again for test dict
# for item in lookList:
#   if (item in testDict):
#     print(item)
#     print(testDict[item])
#     manualCheck = input("    ")
#     if(manualCheck == "l"):
#       finalDict[item] = testDict[item]
#     print(" ")

print(testDict)
#badKeys = ['maple villa long term care centre', "country village homes - woodslee", "peoplecare hilltop manor cambridge", "niagara health system, welland hospital site", "niagara long term care residence", 'the palace', 'meadow park (london)', "chartwell ballycliffe long term care residence", "vision nursing home", "orchard villa", "the grove, arnprior and district nursing home", "extendicare west end villa", 'pinecrest nursing home (bobcaygeon)', 'pinecrest (plantagenet', 'hellenic care for seniors (toronto)', 'west park long term care centre', 'twin lakes terrace long term care community', 'sherwood court long term care centre', 'the henley house', 'st. andrew\'s terrace long term care community', 'valleyview residence', 'southbridge pinewood', 'peoplecare oakcrossing london', 'peoplecare a.r. goudie kitchener', 'providence healthcare', 'shalom manor long term care home', 'st. joseph\'s health centre, guelph', 'villa colombo homes for the aged', 'the perley and rideau veterans\' health centre', 'mount hope centre for long term care', 'extendicare maple view of sault ste. marie', 'the pines', 'wellington terrace long-term care home', 'the elliott long term care residence']
      

In [95]:
with open('finalDict.pkl', 'wb') as f:
  pickle.dump(finalDict, f)

sameList = sameList + list(finalDict.keys())
lookList = []
for item in original:
  if (item not in sameList):
    lookList.append(item)

with open('sameList.pkl', 'wb') as f:
  pickle.dump(sameList, f)


In [ ]:
import difflib

print("We have", len(sameList), "We still need", len(lookList), "original was", len(original))

#Try cloest match
lookList.sort()
for item in lookList:
  possibilities = difflib.get_close_matches(item, homesList)
  print(item)
  print(possibilities)
  manualCheck = input("    ")
  if (len(manualCheck) == 1 and int(manualCheck) <= 2):
    finalDict[item] = possibilities[int(manualCheck)]



In [101]:

finalDict["parisien manor"] = 'parisien manor nursing home cornwall'
finalDict["sherbourne place"] = "drs. paul and john rekai centre"
finalDict["shelburne long term care home"] = "division of provincial nursing home limited partnership - shelburne nursing home"
finalDict["pinecrest (plantagenet)"] = 'pinecrest'
finalDict['belmont long term care facility'] = 'belcrest nursing homes ltd.'
finalDict['southbridge pinewood'] = 'pinewood court'
finalDict['ioof seniors home'] = "odd fellow and rebekah home"
finalDict['mount hope centre for long term care'] = "st. joseph's health care, london - mount hope centre for ltc - st. mary's"
finalDict['brucelea haven long term care home - corporation of the county of bruce'] = "corporation of the county of bruce - brucelea haven ltc home"
finalDict['the pines'] = "the district municipality of muskoka – the pines long term care home"
finalDict['the elliott long term care residence'] = "the elliott community"

# finalDict["seaforth long term care home"] = 
# finalDict['providence healthcare'] = "providence manor"


with open('finalDict.pkl', 'wb') as f:
  pickle.dump(finalDict, f)

sameList = sameList + list(finalDict.keys())
sameList =  list(set(sameList))
lookList = []
for item in original:
  if (item not in sameList):
    lookList.append(item)

with open('sameList.pkl', 'wb') as f:
  pickle.dump(sameList, f)

print("We have", len(sameList), "We still need", len(lookList), "original was", len(original))



We have 511 We still need 2 original was 513
